ref: http://jprogramer.com/ai/3755

In [1]:
import numpy as np
import chainer
import chainer.functions as F
from chainer import optimizers
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
 
 
 
# 数字データをMNISTから読み込む
print("load MNIST dataset")
mnist=fetch_mldata('MNIST original',data_home=".")
x=mnist.data
y=mnist.target
x=x.astype(np.float32)
y=y.astype(np.int32)
 
x /= x.max()    #輝度を揃える
 
#データを「学習データ:テストデータ=9:1」に分ける
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
 
N=y_train.size        #学習データ数
N_test=y_test.size    #テストデータ数
print("train data=%d" % N)
print("test data=%d" % N_test)
 
#[画像数][色数][縦][横]の形式にする
x_train=x_train.reshape((len(x_train),1,28,28))
x_test=x_test.reshape((len(x_test),1,28,28))
 
batchsize=100    #バッチのサイズは100
n_epoch=20        #エポックは20回
n_units=100        #中間層のユニット数は100
 
model=chainer.FunctionSet(            #層の定義
    l1=F.Linear(28*28,n_units),        #入力層:28x28 -> 中間層:n_units
    l2=F.Linear(n_units,10))        #中間層:n_units -> 出力層: 10
 
#順伝搬関数
def forward(x_data,y_data,train=True):
    x=chainer.Variable(x_data)
    t=chainer.Variable(y_data)
    #正規化関数で出力,DropOutで過学習を防ぐ
    h1=F.dropout(F.relu(model.l1(x)),train=train)
    y=model.l2(h1)
    #出力の評価
    if train:
        loss=F.softmax_cross_entropy(y,t)
        return loss
    else:
        acc=F.accuracy(y,t)
        return acc
 
optimizer=optimizers.Adam()
optimizer.setup(model)
 
for epoch in range(1,n_epoch+1):
    print("epoch: %d" % epoch)
    perm=np.random.permutation(N)
    sum_loss=0
    for i in range(0,N,batchsize):
        #学習データの入力と答えを取得
        x_batch=np.asarray(x_train[perm[i:i+batchsize]])
        y_batch=np.asarray(y_train[perm[i:i+batchsize]])
        #勾配をゼロにしておく
        optimizer.zero_grads()
        #順伝搬
        loss=forward(x_batch,y_batch)
        #逆伝搬
        loss.backward()
        #勾配を計算し加える
        optimizer.update()
        #lossの計算
        sum_loss += float(loss.data)*len(y_batch)
    print("train mean loss: %f" % (sum_loss/N))
    #テストデータで計測
    sum_accuracy=0
    for i in range(0,N_test,batchsize):
        #テストデータの入力と答えを取得
        x_batch=np.asarray(x_test[i:i+batchsize])
        y_batch=np.asarray(y_test[i:i+batchsize])
        #順伝搬
        acc=forward(x_batch,y_batch,train=False)
        #正確さを計測
        sum_accuracy += float(acc.data)*len(y_batch)
    print("test accuracy: %f" % (sum_accuracy/N_test))

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


load MNIST dataset
train data=63000
test data=7000
epoch: 1
train mean loss: 0.497663
test accuracy: 0.936714
epoch: 2
train mean loss: 0.274623
test accuracy: 0.950571
epoch: 3
train mean loss: 0.227575
test accuracy: 0.957000
epoch: 4
train mean loss: 0.201663
test accuracy: 0.962429
epoch: 5
train mean loss: 0.183704
test accuracy: 0.964571
epoch: 6
train mean loss: 0.171216
test accuracy: 0.966714
epoch: 7
train mean loss: 0.161155
test accuracy: 0.965143
epoch: 8
train mean loss: 0.151340
test accuracy: 0.967714
epoch: 9
train mean loss: 0.149263
test accuracy: 0.971857
epoch: 10
train mean loss: 0.144648
test accuracy: 0.969857
epoch: 11
train mean loss: 0.140053
test accuracy: 0.969286
epoch: 12
train mean loss: 0.131365
test accuracy: 0.969429
epoch: 13
train mean loss: 0.128589
test accuracy: 0.971286
epoch: 14
train mean loss: 0.125279
test accuracy: 0.971857
epoch: 15
train mean loss: 0.120312
test accuracy: 0.972857
epoch: 16
train mean loss: 0.120268
test accuracy: 0.97042